In [1]:
import pickle

In [2]:
f = open("./pickles/wv_X.pickle","rb")
X = pickle.load(f)
f.close()

In [3]:
f = open("./pickles/author_sentence.pickle","rb")
df = pickle.load(f)
f.close()
df.head()

,sentence,author
0,EUGENIE GRANDET By Honore De Balzac Translate...,"Balzac, Honoré de"
1,"May your name, that of one whose portrait is t...","Balzac, Honoré de"
2,De Balzac.,"Balzac, Honoré de"
3,EUGENIE GRANDET I There are houses in certain ...,"Balzac, Honoré de"
4,"Within these houses there is, perhaps, the sil...","Balzac, Honoré de"


In [4]:
# turn author binerize
#[0,0,0,1,0,0,0,0,.....]
# author_list = list(set(df['author']))
# label = []
# for i in df['author']:
#     label.append([1 if author_list[j] == i else 0 for j in range(len(author_list))])

# [0,15,13,7,3,...]
author_list = list(set(df['author']))
label = []
for i in df['author']:
    label.append(author_list.index(i))

## Test SCV

In [5]:
from sklearn import svm
from sklearn.kernel_approximation import Nystroem

In [6]:
clf = svm.LinearSVC()

In [7]:
breaks = []
for i in range(len(label)-1):
    if label[i] != label[i+1]:
        breaks.append(i)

In [8]:
# TODO: Down Sample the data...
breaks

[82623,
 258888,
 315029,
 424742,
 500036,
 585677,
 674274,
 758273,
 839654,
 910398,
 995015,
 1172958,
 1252179,
 1334049,
 1411109,
 1545376,
 1631822,
 1757397,
 1823025]

In [9]:
from sklearn.utils.random import sample_without_replacement
new_X = []
new_y = []
for i in breaks:
    for j in sample_without_replacement(i, 30000):
        new_X.append(X[j])
        new_y.append(label[j])

In [10]:
feature_map_nystroem = Nystroem(gamma=.2,random_state=1,n_components=300)
data_transformed = feature_map_nystroem.fit_transform(new_X)

In [11]:
feature_map_nystroem.kernel

'rbf'

In [12]:
clf.fit(data_transformed, new_y)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

### Get accuracy

In [13]:
test_X = []
test_y = []
for i in breaks:
    for j in sample_without_replacement(1823024, 3000):
        test_X.append(X[j])
        test_y.append(label[j])

In [14]:
feature_map_nystroem = Nystroem(gamma=.2,random_state=1,n_components=300)
data_transformed = feature_map_nystroem.fit_transform(test_X)

In [15]:
clf.score(data_transformed,test_y)

0.037263157894736845